# Linear Regression Modeling of King County Real Estate Sale Prices
<b>Authors:</b> Aisha Baitemirova-Othman, Angela Kim, Steven Addison, Wahaj Dar
\
<b>Instructor:</b> David Elliott
----

## Overview
This project analyzes residential real estate sales in King County, Washington, and uses the data to create a model that predicts price based on the parameters given.

## Business Problem
Windermere Real Estate, based in Seattle, Washington, wants to better serve home buyers by being able to accurately present a price point using features of a house (ie. number of bedrooms) that buyers are looking for.

## Data Understanding
This dataset contains information about residential real estate sales in King County between May 2014 - May 2015. It includes details such as number of bedrooms and bathrooms, square footage of the home, and various features regarding location.

## Data Preparation

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

!pip install geopy
import geopy
from geopy import distance
import plotly.express as px

import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import RFE

> Here we import our data and skim the first five rows to get a general idea of what the dataframe looks like. We also get an initial look at missing values and datatypes that need to be converted.

In [ ]:
df = pd.read_csv('data/kc_house_data.csv')
display(df.head())
display(df.info())

> Cleaning & preparing the data.

In [ ]:
# Drop the 'id' and 'date' columns
# Fill in missing data
# Convert all object datatype columns to numeric

df['yr_renovated'] = df['yr_renovated'].fillna(0)
df['waterfront'] = df['waterfront'].fillna('NO')
df['waterfront'] = df['waterfront'].str.replace('NO', '0')
df['waterfront'] = df['waterfront'].str.replace('YES', '1')
df['waterfront'] = pd.to_numeric(df['waterfront'])
df['view'] = df['view'].fillna('NONE')
df['grade'] = df['grade'].str.replace('7 Average', '7')
df['grade'] = df['grade'].str.replace('8 Good', '8')
df['grade'] = df['grade'].str.replace('9 Better', '9')
df['grade'] = df['grade'].str.replace('6 Low Average', '6')
df['grade'] = df['grade'].str.replace('10 Very Good', '10')
df['grade'] = df['grade'].str.replace('11 Excellent', '11')
df['grade'] = df['grade'].str.replace('5 Fair', '5')
df['grade'] = df['grade'].str.replace('12 Luxury', '12')
df['grade'] = df['grade'].str.replace('4 Low', '4')
df['grade'] = df['grade'].str.replace('13 Mansion', '13')
df['grade'] = df['grade'].str.replace('3 Poor', '3')
df['grade'] = pd.to_numeric(df['grade'])
if [df[df['sqft_basement'] == '?']]:
    df['sqft_basement'] = df['sqft_living'] - df['sqft_above']
df['sqft_basement'] = pd.to_numeric(df['sqft_basement'])
df['bedrooms'].replace(33, 3, inplace=True)
df['date'] = pd.to_datetime(df['date'])
df['yr_sold'] = df['date'].dt.year
df['house_age'] = df['yr_sold'] - df['yr_built']
df.drop(labels=['id', 'date'], axis=1, inplace=True)

In [ ]:
# One-hot encoding 'condition' and 'view' columns

condition = df[['condition']]
ohe = OneHotEncoder(categories="auto", sparse=False, handle_unknown="ignore")
ohe.fit(condition)
condition_enc = ohe.transform(condition)
condition_enc = pd.DataFrame(condition_enc,
                             columns=['cond_avg','cond_fair','cond_good','cond_poor','cond_verygood'],
                             index=df.index)
df.drop('condition', axis=1, inplace=True)
df = pd.concat([df, condition_enc], axis=1)

view = df[['view']]
ohe.fit(view)
view_enc = ohe.transform(view)
view_enc = pd.DataFrame(view_enc,
                        columns=['view_avg','view_excellent','view_fair','view_good','view_none'],
                        index=df.index)
df.drop('view', axis=1, inplace=True)
df = pd.concat([df, view_enc], axis=1)

In [ ]:
# Create 'distance_from_bellevue' column

bellevue = (47.601, -122.2015)

def distancer(row):
    coords_1 = bellevue
    coords_2 = (row['lat'], row['long'])
    return geopy.distance.distance(coords_1, coords_2).miles

df['distance_from_bellevue'] = df.apply(distancer, axis=1)

# Plot distance map

distancemap = df[df['price'] <= 1000000] 
fig = px.scatter_mapbox(data_frame = distancemap, lat='lat', lon='long', color='price', color_continuous_scale='deep_r')
fig.update_geos(resolution=50)
fig.update_layout(mapbox_style="carto-darkmatter")

In [ ]:
# Examine correlations

corr = df.corr()
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
corr[mask] = np.nan
(corr
 .style
 .background_gradient(cmap='mako', axis=None, vmin=-1, vmax=1)
 .highlight_null(null_color='#f1f1f1')  # Color NaNs grey
 .set_precision(2))

> 'sqft_living' has the highest correlation with 'price' at 0.70. We also see high multicollinearity.

In [ ]:
ix = df.corr().sort_values('price', ascending=False).index
df_sorted = df.loc[:, ix]

plt.figure(figsize=(10,10), dpi=300)
sns.heatmap(df_sorted.corr()[['price']],
            cmap="mako",
            annot=True);

In [ ]:
heatmap = df[['price','sqft_living','grade','sqft_above','distance_from_bellevue','view_none','house_age','zipcode']]
heatmap

In [ ]:
# heatmap for presentation
ix2 = heatmap.corr().sort_values('price', ascending=False).index
df_sorted2 = df.loc[:, ix2]

plt.figure(figsize=(10,10), dpi=300)
sns.heatmap(df_sorted2.corr()[['price']],
            xticklabels=['Price'],
            yticklabels=['Price', 'Sqft Living', 'Grade', 'Sqft Above Ground',
                         'Zipcode', 'House Age', 'No View', 'Distance from Bellevue'],
            cmap="mako",
            annot=True)
plt.yticks(rotation=0);

In [ ]:
# Scatter matrix

scatter_columns = ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
                   'floors', 'waterfront', 'grade', 'sqft_above', 'sqft_basement',
                   'yr_built', 'zipcode', 'lat', 'long', 'sqft_living15','sqft_lot15']

df_scatter = df[scatter_columns]

sns.pairplot(df_scatter, corner=True);

> Scatter matrix shows many non-normal distributions.

## Inferential Modeling

In [ ]:
# Analyzing OLS results

outcome = 'price'
dfx = df.drop('price', axis=1)
predictors = '+'.join(dfx.columns)
formula = outcome + '~' + predictors
model = ols(formula=formula, data=df).fit()
model.summary()

> The p-values for 'floors' and 'sqft_lot15' are not statistically significant. JB is very high, indicating non-normal distributions. There is strong multicollinearity.

> Previously, we saw that 'price' and 'sqft_living' have the strongest correlation, but the scatter matrix reveals that they are not normally distributed.

In [ ]:
# OLS between 'price' and 'sqft_living'
f = 'price~sqft_living'
model = ols(f, df).fit()
model.summary()

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, 'sqft_living', fig=fig);

> Plots show heteroscedasticity.

In [ ]:
f = 'price~distance_from_bellevue'
model = ols(f, df).fit()
model.summary()

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, "distance_from_bellevue", fig=fig)
plt.show()

In [ ]:
# Normalizing distribution using log transformation

df0 = df.copy()
df0['price_log'] = np.log(df0['price'])
df0['sqft_living_log'] = np.log(df0['sqft_living'])
df0 = df0.drop(['price', 'sqft_living'], axis=1)

# OLS between 'price_log' and 'sqft_living_log'

f = 'price_log~sqft_living_log'
model = ols(f, df0).fit()
model.summary()

In [ ]:
fig = plt.figure(figsize=(15,8))
fig = sm.graphics.plot_regress_exog(model, 'sqft_living_log', fig=fig);

> When 'price' and 'sqft_living' undergo log transformation, they are more normally distributed and more homoscedastic, making them better for modeling.

## Predictive Modeling

<b>Baseline Model & First Simple Linear Regression Model</b>

In [ ]:
# df0 has original price & sqft_living removed, has price_log & sqft_living_log
X = df0[['sqft_living_log']]
y = df0[['price_log']]
X_train, X_test, y_train, y_test = train_test_split(X, y)

# baseline
baseline = DummyRegressor()
baseline.fit(X_train, y_train)
print('Baseline Train R\u00b2:', baseline.score(X_train, y_train))
print('Baseline Test R\u00b2:', baseline.score(X_test, y_test))
print()

# simple lr
lr = LinearRegression()
lr.fit(X_train, y_train)
y_hat_train = lr.predict(X_train)
y_hat_test = lr.predict(X_test)
train_rmse = mse(y_train, y_hat_train, squared=False)
test_rmse = mse(y_test, y_hat_test, squared=False)
print('Simple LR Train R\u00b2:', lr.score(X_train, y_train))
print('Simple LR Test R\u00b2:', lr.score(X_test, y_test))
print('Simple LR Train RMSE:', train_rmse)
print('Simple LR Test RMSE:', test_rmse)
y_test_pred = lr.predict(X_test)
plt.scatter(y_test_pred, y_test)
plt.scatter(y_test, y_test);

<b>First Multiple Linear Regression Model</b>
\
Model with all untouched predictor variables.

In [ ]:
# df1 = original df
df1 = df.copy()
X1 = df1.drop(['price'], axis=1)
y1 = df1[['price']]
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1)
lr1 = LinearRegression()
lr1.fit(X1_train, y1_train)
y1_hat_train = lr1.predict(X1_train)
y1_hat_test = lr1.predict(X1_test)
train1_rmse = mse(y1_train, y1_hat_train, squared=False)
test1_rmse = mse(y1_test, y1_hat_test, squared=False)
print('LR1 Train R\u00b2:', lr1.score(X1_train, y1_train))
print('LR1 Test R\u00b2:', lr1.score(X1_test, y1_test))
print('LR1 Train RMSE:', train1_rmse)
print('LR1 Test RMSE:', test1_rmse)
y1_test_pred = lr1.predict(X1_test)
plt.scatter(y1_test_pred, y1_test)
plt.scatter(y1_test, y1_test);

> There are outliers in our dataset affecting our model.

In [ ]:
plt.figure(figsize=(15,8))
sns.histplot(df['price'], kde=True);

In [ ]:
plt.figure(figsize=(15,8))
sns.histplot(df['sqft_living'], kde=True);

In [ ]:
plt.figure(figsize =(15,8))
sns.histplot(df['distance_from_bellevue'], kde=True);

<b>Second Multiple Linear Regression Model</b>
\
Model with price, sqft_living, distance_from_bellevue, and other continuous variable outliers removed.

In [ ]:
price_low = df1["price"].quantile(0.023)
price_hi = df1["price"].quantile(0.977)

sqft_low = df1['sqft_living'].quantile(0.023)
sqft_hi = df1['sqft_living'].quantile(0.977)

distance_hi = df1['distance_from_bellevue'].quantile(0.99)

df2 = df1.copy()
df2 = df2[(df2["price"] < price_hi) & (df2["price"] > price_low)]
df2 = df2[(df2['sqft_living'] < sqft_hi) & (df2['sqft_living'] > sqft_low)]
df2 = df2[(df2['distance_from_bellevue'] < distance_hi)]

In [ ]:
X2 = df2.drop(['price'], axis=1)
y2 = df2[['price']]
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2)
lr2 = LinearRegression()
lr2.fit(X2_train, y2_train)
y2_hat_train = lr2.predict(X2_train)
y2_hat_test = lr2.predict(X2_test)
train2_rmse = mse(y2_train, y2_hat_train, squared=False)
test2_rmse = mse(y2_test, y2_hat_test, squared=False)
print('LR2 Train R\u00b2:', lr2.score(X2_train, y2_train))
print('LR2 Test R\u00b2:', lr2.score(X2_train, y2_train))
print('LR2 Train RMSE:', train2_rmse)
print('LR2 Test RMSE:', test2_rmse)
y2_test_pred = lr2.predict(X2_test)
plt.scatter(y2_test_pred, y2_test)
plt.scatter(y2_test, y2_test);

<b>Third Multiple Linear Regression Model</b>
\
Second model with log transformed price, sqft_living, and distance_from_bellevue, and other continuous variables.

In [ ]:
df3 = df2.copy()
df3['price_log'] = np.log(df2['price'])
df3['sqft_living_log'] = np.log(df3['sqft_living'])
df3['distance_from_bellevue_log'] = np.log(df3['distance_from_bellevue'])
df3['sqft_lot_log'] = np.log(df2['sqft_lot'])
df3 = df3.drop(['price', 'sqft_living', 'distance_from_bellevue','sqft_lot'], axis=1)

In [ ]:
X3 = df3.drop(['price_log'], axis=1)
y3 = df3[['price_log']]
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3)
lr3 = LinearRegression()
lr3.fit(X3_train, y3_train)
y3_hat_train = lr3.predict(X3_train)
y3_hat_test = lr3.predict(X3_test)
train3_rmse = mse(y3_train, y3_hat_train, squared=False)
test3_rmse = mse(y3_test, y3_hat_test, squared=False)
print('LR3 Train R\u00b2:', lr3.score(X3_train, y3_train))
print('LR3 Test R\u00b2:', lr3.score(X3_test, y3_test))
print('LR3 Train RMSE:', train3_rmse)
print('LR3 Test RMSE:', test3_rmse)
y3_test_pred = lr3.predict(X3_test)
plt.scatter(y3_test_pred, y3_test)
plt.scatter(y3_test, y3_test);

<b>Fourth Multiple Linear Regression Model</b>
\
Third model with multicollinear variables removed.

In [ ]:
corr = df3.corr()
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
corr[mask] = np.nan
(corr
 .style
 .background_gradient(cmap='mako', axis=None, vmin=-1, vmax=1)
 .highlight_null(null_color='#f1f1f1')  # Color NaNs grey
 .set_precision(2))

In [ ]:
df4 = df3[['price_log', 'sqft_living_log', 'distance_from_bellevue_log', 'bedrooms', 'sqft_lot_log',
          'waterfront', 'yr_renovated', 'house_age', 'view_none']]

In [ ]:
corr = df4.corr()
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
corr[mask] = np.nan
(corr
 .style
 .background_gradient(cmap='mako', axis=None, vmin=-1, vmax=1)
 .highlight_null(null_color='#f1f1f1')  # Color NaNs grey
 .set_precision(2))

In [ ]:
X4 = df4.drop(['price_log'], axis=1)
y4 = df4[['price_log']]
X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y4)
lr4 = LinearRegression()
lr4.fit(X4_train, y4_train)
y4_hat_train = lr4.predict(X4_train)
y4_hat_test = lr4.predict(X4_test)
train4_rmse = mse(y4_train, y4_hat_train, squared=False)
test4_rmse = mse(y4_test, y4_hat_test, squared=False)
print('LR4 Train R\u00b2:', lr4.score(X4_train, y4_train))
print('LR4 Test R\u00b2:', lr4.score(X4_test, y4_test))
print('LR4 Train RMSE:', train4_rmse)
print('LR4 Test RMSE:', test4_rmse)
y4_test_pred = lr4.predict(X4_test)
plt.scatter(y4_test_pred, y4_test)
plt.scatter(y4_test, y4_test);

<b>Fifth Multiple Linear Regression Model</b>
\
Fourth model with several predictor variables scaled.

In [ ]:
df5 = df4.copy()
col_names = ['sqft_living_log', 'distance_from_bellevue_log', 'sqft_lot_log',
            'yr_renovated','house_age']
features = df5[col_names]
scaler = StandardScaler().fit(features.values)
features = scaler.transform(features.values)
df5[col_names] = features

In [ ]:
X5 = df5.drop(['price_log'], axis=1)
y5 = df5[['price_log']]
X5_train, X5_test, y5_train, y5_test = train_test_split(X5, y5)
lr5 = LinearRegression()
lr5.fit(X5_train, y5_train)
y5_hat_train = lr5.predict(X5_train)
y5_hat_test = lr5.predict(X5_test)
train5_rmse = mse(y5_train, y5_hat_train, squared=False)
test5_rmse = mse(y5_test, y5_hat_test, squared=False)
print('LR5 Train R\u00b2:', lr5.score(X5_train, y5_train))
print('LR5 Test R\u00b2:', lr5.score(X5_test, y5_test))
print('LR5 Train RMSE:', train5_rmse)
print('LR5 Test RMSE:', test5_rmse)
y5_test_pred = lr5.predict(X5_test)
plt.scatter(y5_test_pred, y5_test)
plt.scatter(y5_test, y5_test);

In [ ]:
selector = RFE(lr5, n_features_to_select=4)
selector = selector.fit(X5, y5)
print(selector.support_)
display(X5)

> Selector selects sqft_living_log, distance_from_bellevue_log, waterfront, & view_none.

<b>Sixth Multiple Linear Regression Model</b>
\
Fifth model using recursive feature elimination.

In [ ]:
df6 = df5.copy()
X6 = df6[['sqft_living_log', 'distance_from_bellevue_log', 'waterfront', 'view_none']]
y6 = df6[['price_log']]
X6_train, X6_test, y6_train, y6_test = train_test_split(X6, y6)
lr6 = LinearRegression()
lr6.fit(X6_train, y6_train)
y6_hat_train = lr6.predict(X6_train)
y6_hat_test = lr6.predict(X6_test)
train6_rmse = mse(y6_train, y6_hat_train, squared=False)
test6_rmse = mse(y6_test, y6_hat_test, squared=False)
print('LR6 Train R\u00b2:', lr6.score(X6_train, y6_train))
print('LR6 Test R\u00b2:', lr6.score(X6_test, y6_test))
print('LR6 Train RMSE:', train6_rmse)
print('LR6 Test RMSE:', test6_rmse)
y6_test_pred = lr6.predict(X6_test)
plt.scatter(y6_test_pred, y6_test)
plt.scatter(y6_test, y6_test);

<b>Seventh Multiple Linear Regression Model</b>
\
Sixth model using stepwise selection to choose significant features.

In [ ]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

result = stepwise_selection(X5, y5, verbose=True)
print('resulting features:')
print(result)

In [ ]:
df7 = df6.copy()
X7 = df7[['sqft_living_log', 'distance_from_bellevue_log', 'waterfront', 'view_none']]
y7 = df7[['price_log']]
X7_train, X7_test, y7_train, y7_test = train_test_split(X7, y7)
lr7 = LinearRegression()
lr7.fit(X7_train, y7_train)
y7_hat_train = lr7.predict(X7_train)
y7_hat_test = lr7.predict(X7_test)
train7_rmse = mse(y7_train, y7_hat_train, squared=False)
test7_rmse = mse(y7_test, y7_hat_test, squared=False)
print('LR7 Train R\u00b2:', lr7.score(X7_train, y7_train))
print('LR7 Test R\u00b2:', lr7.score(X7_test, y7_test))
print('LR7 Train RMSE:', train7_rmse)
print('LR7 Test RMSE:', test7_rmse)
y7_test_pred = lr7.predict(X7_test)
plt.scatter(y7_test_pred, y7_test)
plt.scatter(y7_test, y7_test);

## Conclusions

In [ ]:
baseline = baseline.score(X_train, y_train)
simple = lr.score(X_train, y_train)
first = lr1.score(X1_train, y1_train)
second = lr2.score(X2_train, y2_train)
third = lr3.score(X3_train, y3_train)
fourth = lr4.score(X4_train, y4_train)
fifth = lr5.score(X5_train, y5_train)
sixth = lr6.score(X6_train, y6_train)
seventh = lr7.score(X7_train, y7_train)

barchart = pd.DataFrame({'Model':['baseline', 'simple', 'first', 'second',
                                  'third', 'fourth', 'fifth', 'sixth', 'seventh'],
                         'R\u00b2':[baseline, simple, first, second, third, fourth, fifth, sixth, seventh]})
plt.figure(figsize=(15,10), dpi=300)
ax = sns.barplot(x=barchart['Model'], y=barchart['R\u00b2'], palette="mako")
plt.title("Models with their R\u00b2 Values", fontsize=20)
ax.set_xlabel('Model Number', fontsize=16)
ax.set_ylabel('R\u00b2 Value', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12);

In [ ]:
lr6.coef_

In [ ]:
plotdf = pd.DataFrame({'Predictor':['Sqft Living', 'Distance From Bellevue', 'Waterfront Property', 'No View'],
                       'Coefficient':[0.23807778, -0.20044333,  0.42040876, -0.20787165]})
plotdf = plotdf.sort_values(by=['Coefficient'])

In [ ]:
plt.figure(figsize=(16,10), dpi=300)
ax = sns.barplot(x=plotdf['Predictor'], y=plotdf['Coefficient'], palette="mako")
plt.title('Top House Price Predictors', fontsize=20)
ax.set_xlabel('Predictor', fontsize=16)
ax.set_ylabel('Coefficient', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12);